In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


## Explore a base model that only uses microdensities to determine the next microdensity.
Thus, to increase our data:  
- We have 39 instances per county
- We can use the first 23 to calculate the 24, then 2-25 to calculate 26, and so on.
- We then have trained a model that can predict the next microdensity based on previous instances.

- As a later stage we can include census information as an extra filtering technique

In [3]:
df = pd.read_csv(
    "../data/processed/county_data.csv",
    usecols=[
        "cfips",
        "first_day_of_month",
        "active",
        "POPESTIMATE2021",
        "microbusiness_density",
    ],
)
test = pd.read_csv("../data/raw/test.csv")

In [4]:
df.head()

,cfips,first_day_of_month,microbusiness_density,active,POPESTIMATE2021
0,1001,2019-08-01,3.007682,1249,59095
1,1001,2019-09-01,2.884870,1198,59095
2,1001,2019-10-01,3.055843,1269,59095
3,1001,2019-11-01,2.993233,1243,59095
4,1001,2019-12-01,2.993233,1243,59095


In [5]:
df["cfips"].value_counts()

1001     39
39133    39
39089    39
39091    39
39093    39
         ..
21113    39
21115    39
21117    39
21119    39
56045    39
Name: cfips, Length: 3135, dtype: int64

### Create the timeseries

In [6]:
# Create an empty list to store the new time series
time_series = []

# Loop over each cfips
for cfips in df["cfips"].unique():
    # Filter the data for the current cfips
    df_cfips = df[df["cfips"] == cfips].reset_index(drop=True)
    # Loop over the time series from 0 to 14 (inclusive)
    for i in range(0, 15):
        # Create a new time series with the values for the current cfips and range of first_day_of_month
        time_series.append(
            {
                "cfips": cfips,
                "POPESTIMATE2021": df_cfips.loc[0, "POPESTIMATE2021"],
                "active": df_cfips.loc[0, "active"],
                "microbusiness_density": df_cfips.loc[
                    i : i + 23, "microbusiness_density"
                ].values.tolist(),
                "target": df_cfips.loc[i + 24, "microbusiness_density"],
                "target_date": df_cfips.loc[i + 24, "first_day_of_month"],
            }
        )

# Create a new dataframe with the list of time series
new_df = pd.DataFrame(time_series)

In [7]:
new_df.head()

,cfips,POPESTIMATE2021,active,microbusiness_density,target,target_date
0,1001,59095,1249,"[3.0076818, 2.8848701, 3.0558431, 2.9932332, 2...",3.219917,2021-08-01
1,1001,59095,1249,"[2.8848701, 3.0558431, 2.9932332, 2.9932332, 2...",3.186722,2021-09-01
2,1001,59095,1249,"[3.0558431, 2.9932332, 2.9932332, 2.96909, 2.9...",3.203320,2021-10-01
3,1001,59095,1249,"[2.9932332, 2.9932332, 2.96909, 2.9093256, 2.9...",3.200948,2021-11-01
4,1001,59095,1249,"[2.9932332, 2.96909, 2.9093256, 2.9332314, 3.0...",3.286307,2021-12-01


### Make it a dataframe

In [8]:
# convert the list of microbusiness_density to a numpy array and then each into a column
new_df["microbusiness_density"] = new_df["microbusiness_density"].apply(
    lambda x: np.array(x)
)
new_df = pd.concat(
    [
        new_df.drop(["microbusiness_density"], axis=1),
        new_df["microbusiness_density"].apply(pd.Series),
    ],
    axis=1,
)
new_df.head()

,cfips,POPESTIMATE2021,active,target,target_date,0,1,2,3,4,...,14,15,16,17,18,19,20,21,22,23
0,1001,59095,1249,3.219917,2021-08-01,3.007682,2.884870,3.055843,2.993233,2.993233,...,3.193804,3.038416,3.002558,2.947244,3.106106,3.144043,3.224659,3.227030,3.222288,3.210433
1,1001,59095,1249,3.186722,2021-09-01,2.884870,3.055843,2.993233,2.993233,2.969090,...,3.038416,3.002558,2.947244,3.106106,3.144043,3.224659,3.227030,3.222288,3.210433,3.219917
2,1001,59095,1249,3.203320,2021-10-01,3.055843,2.993233,2.993233,2.969090,2.909326,...,3.002558,2.947244,3.106106,3.144043,3.224659,3.227030,3.222288,3.210433,3.219917,3.186722
3,1001,59095,1249,3.200948,2021-11-01,2.993233,2.993233,2.969090,2.909326,2.933231,...,2.947244,3.106106,3.144043,3.224659,3.227030,3.222288,3.210433,3.219917,3.186722,3.203320
4,1001,59095,1249,3.286307,2021-12-01,2.993233,2.969090,2.909326,2.933231,3.000167,...,3.106106,3.144043,3.224659,3.227030,3.222288,3.210433,3.219917,3.186722,3.203320,3.200948


In [9]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47025 entries, 0 to 47024
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cfips            47025 non-null  int64  
 1   POPESTIMATE2021  47025 non-null  int64  
 2   active           47025 non-null  int64  
 3   target           47025 non-null  float64
 4   target_date      47025 non-null  object 
 5   0                47025 non-null  float64
 6   1                47025 non-null  float64
 7   2                47025 non-null  float64
 8   3                47025 non-null  float64
 9   4                47025 non-null  float64
 10  5                47025 non-null  float64
 11  6                47025 non-null  float64
 12  7                47025 non-null  float64
 13  8                47025 non-null  float64
 14  9                47025 non-null  float64
 15  10               47025 non-null  float64
 16  11               47025 non-null  float64
 17  12          

### Persist

In [10]:
new_df.to_csv("../data/processed/timeseries_15_rows_per_county.csv", index=False)

## Machine Learning

In [ ]:
test["first_day_of_month"] = pd.to_datetime(test["first_day_of_month"])
test.sort_values(["cfips", "first_day_of_month"], inplace=True)

In [ ]:
ids = df["cfips"].unique()
# create X, which is df_new without the target column
X = new_df.drop(["target", "target_date", "cfips", "POPESTIMATE2021", "active"], axis=1)
y = new_df["target"]
grps = new_df["cfips"]
threshold = 0.1

In [ ]:
# import group kfold
from sklearn.model_selection import GroupKFold

# import xgboost predictor
from xgboost import XGBRegressor

### Kfold groups for validating results on XGboost

In [ ]:
# create a group kfold object
group_kfold = GroupKFold(n_splits=5)
print(X.shape, y.shape, ids.shape)
models_scores = {}
# split the data and targets
for split, (train_index, valid_index) in enumerate(
    group_kfold.split(X, y, groups=grps)
):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    print(f"Split {split}:")
    # predict with xgboost
    xgb = XGBRegressor()
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_valid)
    # calculate the mean absolute error
    mae = np.mean(np.abs(y_pred - y_valid))
    print(f"MAE: {mae}")
    # calculate sMAPE
    smape = np.mean(np.abs(y_pred - y_valid) / (np.abs(y_pred) + np.abs(y_valid)))
    print(f"sMAPE: {smape}")
    # calculate the percentage of predictions that are within the  threshold

    pct_within_threshold = np.mean((smape) <= threshold) * 100
    print(f"Percentage within threshold: {pct_within_threshold}")
    models_scores[split] = mae

### Results Kfold

In [ ]:
models_scores

### Train whole dataset XGboosting

In [ ]:
# train model on whole dataset
xgb = XGBRegressor()
xgb.fit(X, y)

In [ ]:
ids = test["cfips"].unique()
predictions = np.zeros((len(ids), 8))
test_df = new_df[new_df["target_date"] == new_df["target_date"].max()].drop(
    ["target", "target_date", "POPESTIMATE2021", "active", "cfips"], axis=1
)

### Predictions

In [ ]:
# loop over the dataset and make predictions adding the last prediction as input for the next point
for i in range(8):
    predictions[:, i] = xgb.predict(test_df)
    # print(test_df.iloc[1, :])
    test_df[i + 24] = predictions[:, i]
    test_df.drop(0, axis=1, inplace=True)
    # rename the columns to 0:23
    test_df.columns = range(0, 24)
    # print(test_df.iloc[1, :])

In [ ]:
predictions.reshape(-1).shape

In [ ]:
test.shape

### Submission file

In [ ]:
# submission = test_df[:, -8:]
test["microbusiness_density"] = predictions.reshape(-1)

In [ ]:
test.head(8)

In [ ]:
result = test[["row_id", "microbusiness_density"]]
result.to_csv("../data/submissions/ml_test_1.csv", index=False)

In [ ]:
import tensorflow as tf

In [ ]:
def build_model():

    inp = tf.keras.Input(shape=(24, 1))

    x = tf.keras.layers.GRU(units=128, return_sequences=False)(inp)
    x = tf.keras.layers.Dense(1, activation="linear")(x)
    model = tf.keras.Model(inputs=inp, outputs=x)

    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    loss = tf.keras.losses.MeanSquaredError()
    model.compile(loss=loss, optimizer=opt)

    return model

In [ ]:
model = build_model()

In [ ]:
model.fit(X, y, epochs=25)

In [ ]:
model.save_weights("../models/ml_test_1.h5")

## train the model but only with highly populated counties

In [ ]:
model = build_model()

In [ ]:
# find the 50% percentile based on POPESTIMATE2021
subset_df = new_df[new_df["POPESTIMATE2021"] > new_df["POPESTIMATE2021"].quantile(0.5)]
X_subset = subset_df.drop(
    ["target", "target_date", "cfips", "POPESTIMATE2021", "active"], axis=1
)
y_subset = subset_df["target"]

In [ ]:
# find the 50% percentile based on POPESTIMATE2021
subset_df = new_df[new_df["POPESTIMATE2021"] > new_df["POPESTIMATE2021"].quantile(0.5)]
X_subset = subset_df.drop(
    ["target", "target_date", "cfips", "POPESTIMATE2021", "active"], axis=1
)
y_subset = subset_df["target"]

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices("GPU")))
# train with gpu
result = None
with tf.device("/GPU:0"):
    result = model.fit(X_subset, y_subset, epochs=25)

In [ ]:
# plot the loss
fig = plt.figure(figsize=(10, 5))
plt.plot(result.history["loss"])
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()